# Econ Problem Set 1

## Ari Boyarsky

In [6]:
import numpy as np
from quantecon import tauchen

UntypedAttributeError: Failed at nopython (nopython frontend)
Unknown attribute 'iinfo' of type Module(<module 'numpy' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\numpy\\__init__.py'>)
File "..\..\..\..\..\..\..\..\ProgramData\Anaconda3\lib\site-packages\quantecon\util\numba.py", line 94
[1] During: typing of get attribute at C:\ProgramData\Anaconda3\lib\site-packages\quantecon\util\numba.py (94)

### Problem 1
State Variables: $z_t, k_t$

### Problem 2
State Variables: $c_t, k_{t+1}$

### Problem 3
Bellman: $V(z_t, k_t) = \max u(c_t) + \beta \mathbb{E}V(z_{t+1}, k_{t+1})$ s.t. $zk^\alpha - c - k' + (1-\delta)k = 0$

### Problem 4
### Solution

$\mathcal{L} =\sum_t \beta^t u(c_t) - \sum_t \lambda_t\beta^t[z_tk_t^\alpha - c_t - k_{t+1} + (1-\delta)k_t]$

Then, 

$\mathcal{L} =\sum_t \beta^t u(c) - \sum_t \lambda_t\beta^t[zk^\alpha - c - k' + (1-\delta)k]$

FOC:

$\beta^tc^{-\gamma} = - \lambda_t$
$\beta^tc^{-\gamma} = 

In [9]:
gamma = 0.5
beta = 0.96
delta = 0.05
alpha = 0.4
sigma = 0.2

In [16]:
# Set up Grid

'''
------------------------------------------------------------------------
Create Grid for State Space    
------------------------------------------------------------------------
lb_k      = scalar, lower bound of capital grid
ub_k      = scalar, upper bound of capital grid 
lb_z      = scalar, lower bound of z grid
ub_z      = scalar, upper bound of z grid  
------------------------------------------------------------------------
'''

lb_z = 0.2 
ub_z = 1.0 
lb_k = 0.2
ub_k = 10
size_z = 10  # Number of grid points
size_k = 10  # Number of grid points
z_grid = np.linspace(lb_z, ub_z, size_z)
k_grid = np.linspace(lb_k, ub_k, size_k)

'''
------------------------------------------------------------------------
Create grid of current utility values    
------------------------------------------------------------------------
'''

C = np.zeros((size_k, size_z)) 
for i in range(size_z): # loop over w
    for j in range(size_k): # loop over w'
        C[i, j] = w_grid[i] - w_grid[j] / R # note that if w'>w, consumption negative
# replace 0 and negative consumption with a tiny value 
# This is a way to impose non-negativity on cons
C[C<=0] = 1e-15
if sigma == 1:
    U = np.log(C)
else:
    U = (C ** (1 - sigma)) / (1 - sigma)
U[C<0] = -9999999



SyntaxError: invalid syntax (<ipython-input-16-19b8bb242091>, line 24)

In [18]:
'''
------------------------------------------------------------------------
Value Function Iteration    
------------------------------------------------------------------------
VFtol     = scalar, tolerance required for value function to converge
VFdist    = scalar, distance between last two value functions
VFmaxiter = integer, maximum number of iterations for value function
V         = vector, the value functions at each iteration
Vmat      = matrix, the value for each possible combination of w and w'
Vstore    = matrix, stores V at each iteration 
VFiter    = integer, current iteration number
TV        = vector, the value function after applying the Bellman operator
PF        = vector, indicies of choices of w' for all w 
VF        = vector, the "true" value function
------------------------------------------------------------------------
'''
VFtol = 1e-4 
VFdist = 7.0 
VFmaxiter = 3000 
V = np.zeros((size_k, size_z)) # initial guess at value function
Vmat = np.zeros((size_k, size_k, size_z)) # initialize Vmat matrix
Vstore = np.zeros((size_k, size_z, VFmaxiter)) #initialize Vstore array
VFiter = 1 
while VFdist > VFtol and VFiter < VFmaxiter:  
    for i in range(size_k): # loop over k
        for j in range(size_k): # loop over w'
            for n in range(size_z):
                Vmat[i, j, n] = U[i, j] + beta * V[j, n] 
      
    Vstore[:, VFiter] = V.reshape(size_w,) # store value function at each iteration for graphing later
    TV = Vmat.max(1) # apply max operator to Vmat (to get V(w))
    PF = np.argmax(Vmat, axis=1)
    VFdist = (np.absolute(V - TV)).max()  # check distance
    V = TV
    VFiter += 1 
    


if VFiter < VFmaxiter:
    print('Value function converged after this many iterations:', VFiter)
else:
    print('Value function did not converge')            


VF = V # solution to the functional equation

NameError: name 'U' is not defined